# Fifa World Cup 2018 Prediction Model

### Basis of Predictions

A brief description of the prediction process. Did you use:

* Model results for prediction, 
* Other approaches (e.g. bookie odds) 
* A combination of the two. 

If you used a combination or just another approach, please indicate what other method was used and which games it was used for.

### Data Used

A brief description of the data used in the notebook, including the features and labels. 

### Models

A brief description of the model(s) used for final predictions. Also include the other models that were tested in the notebook.

### Model Results

An overview of your model results on the training and validation sets.


### Additional Info

Any additional things you may want to add.

## 1. Import Libraries

In [1]:
import pandas as pd
from collections import namedtuple
from itertools import chain, groupby
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
from collections import namedtuple
from itertools import chain, groupby, combinations
import psycopg2

## 2. Import Data 

In [2]:
# Connect to RDS Instance Database

# Do not change
rds_endpoint = 'yevashanrdsinstance.cnzbp4ndrpos.eu-west-1.rds.amazonaws.com'
rds_user = 'Yevashan12'
rds_psswrd = 'EdsaYev12!'
port = 5432
rds_database = 'yevashanWorldCupDB'
try:
    conn = psycopg2.connect("dbname='{:s}' user='{:s}' host='{:s}' password='{:s}'".format(rds_database, rds_user, rds_endpoint, rds_psswrd))
    print("Connected to the database")
except:
    print("Unable to connect to the database, please check your settings")

Connected to the database


In [3]:
#Retreive the tables that will be used
match_results = pd.read_sql_query("SELECT * FROM historic_team_results", con=conn)
match_date_type = pd.read_sql_query("SELECT * FROM historic_df", con=conn)
teams = pd.read_sql_query("SELECT * FROM teams", con=conn)
#teams = pd.read_sql_query("SELECT * FROM teams", con=conn)
#leaving out squads for now
#leaving current WC infor and fixtures. Will add in when doing predictions

In [4]:
match_results.head()

,id,historic_match_id,team_id,goals
0,1,1,9,6.0
1,2,1,25,3.0
2,3,2,12,2.0
3,4,2,26,5.0
4,5,3,10,2.0


In [5]:
match_date_type.head()

,id,date,cup_name
0,1,1956-11-11,FIFA WC 1958 Prel. Comp. UEFA
1,2,1956-12-05,FIFA WC 1958 Prel. Comp. UEFA
2,3,1956-11-27,Olympic Football Tournament Melbourne 1956
3,4,1956-06-03,OFT > 1956 > Preliminary Round
4,5,1956-06-10,OFT > 1956 > Preliminary Round


In [6]:
teams.head()

,id,short_name,long_name,fifa_ranking,overall_rating,att,mid,def,squad_rating
0,1,RUS,Russia,66.0,79.0,79.0,77.0,75.0,0.0
1,2,KSA,Saudi Arabia,67.0,72.0,71.0,72.0,71.0,0.0
2,3,EGY,Egypt,46.0,76.0,75.0,77.0,74.0,0.0
3,4,URU,Uruguay,17.0,80.0,86.0,77.0,80.0,0.0
4,5,POR,Portugal,4.0,84.0,88.0,83.0,82.0,0.0


## 3. Clean Data

In [17]:
# Make the match results table have 1 row per match
df = pd.merge(match_results,match_results,on='historic_match_id',suffixes=('_t1', '_t2'))
df.drop_duplicates(subset=['id_t1','historic_match_id'],keep='first',inplace=True)
df.drop_duplicates(subset=['historic_match_id'],keep='last',inplace=True)
df.head(10)

,id_t1,historic_match_id,team_id_t1,goals_t1,id_t2,team_id_t2,goals_t2
2,2,1,25,3.0,1,9,6.0
6,4,2,26,5.0,3,12,2.0
10,6,3,32,0.0,5,10,2.0
14,8,4,24,0.0,7,32,2.0
18,10,5,24,2.0,9,32,0.0
22,12,6,22,1.0,11,19,1.0
26,14,7,13,0.0,13,11,0.0
30,16,8,22,0.0,15,11,2.0
34,18,9,17,1.0,17,11,0.0
38,20,10,13,4.0,19,19,3.0


In [18]:
#Join dates and match type
df = df.merge(match_date_type,how='left',left_on='historic_match_id', right_on ='id' )



In [19]:
# Join longer team names
df = df.merge(teams,how='left',left_on='team_id_t1',right_on='id',suffixes='_1')
df = df.merge(teams,how='left',left_on='team_id_t2',right_on='id',suffixes='_2')
df  =df.drop(['id_t1','historic_match_id','team_id_t1','id_t2','team_id_t2','id_','id1','short_name_',
            'short_name2','id'],axis=1)
df

,goals_t1,goals_t2,date,cup_name,long_name_,fifa_ranking_,overall_rating_,att_,mid_,def_,squad_rating_,long_name2,fifa_ranking2,overall_rating2,att2,mid2,def2,squad_rating2
0,3.0,6.0,1956-11-11,FIFA WC 1958 Prel. Comp. UEFA,Belgium,3.0,83.0,85.0,83.0,83.0,0.0,France,7.0,85.0,86.0,84.0,82.0,0.0
1,5.0,2.0,1956-12-05,FIFA WC 1958 Prel. Comp. UEFA,England,13.0,81.0,83.0,81.0,80.0,0.0,Denmark,12.0,78.0,76.0,79.0,76.0,0.0
2,0.0,2.0,1956-11-27,Olympic Football Tournament Melbourne 1956,Japan,60.0,75.0,76.0,78.0,73.0,0.0,Australia,40.0,72.0,69.0,72.0,70.0,0.0
3,0.0,2.0,1956-06-03,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,0.0,Japan,60.0,75.0,76.0,78.0,73.0,0.0
4,2.0,0.0,1956-06-10,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,0.0,Japan,60.0,75.0,76.0,78.0,73.0,0.0
5,1.0,1.0,1956-02-26,Friendlies,Mexico,15.0,79.0,81.0,79.0,77.0,0.0,Costa Rica,25.0,73.0,69.0,73.0,73.0,0.0
6,0.0,0.0,1956-02-28,Friendlies,Argentina,5.0,84.0,86.0,82.0,81.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0
7,0.0,2.0,1956-03-04,Friendlies,Mexico,15.0,79.0,81.0,79.0,77.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0
8,1.0,0.0,1956-03-06,Friendlies,Brazil,2.0,86.0,87.0,85.0,84.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0
9,4.0,3.0,1956-03-06,Friendlies,Argentina,5.0,84.0,86.0,82.0,81.0,0.0,Costa Rica,25.0,73.0,69.0,73.0,73.0,0.0


In [20]:
# Check for Duplicates
#Check for Nulss

In [24]:
# convert date strings to datetime objects and create 'year' field
df.loc[:,'date'] = pd.to_datetime(df.loc[:,'date'].astype(str), format='%Y-%m-%d')
df['year'] = [x.year for x in df['date']]
df

,goals_t1,goals_t2,date,cup_name,long_name_,fifa_ranking_,overall_rating_,att_,mid_,def_,squad_rating_,long_name2,fifa_ranking2,overall_rating2,att2,mid2,def2,squad_rating2,year
0,3.0,6.0,1956-11-11,FIFA WC 1958 Prel. Comp. UEFA,Belgium,3.0,83.0,85.0,83.0,83.0,0.0,France,7.0,85.0,86.0,84.0,82.0,0.0,1956
1,5.0,2.0,1956-12-05,FIFA WC 1958 Prel. Comp. UEFA,England,13.0,81.0,83.0,81.0,80.0,0.0,Denmark,12.0,78.0,76.0,79.0,76.0,0.0,1956
2,0.0,2.0,1956-11-27,Olympic Football Tournament Melbourne 1956,Japan,60.0,75.0,76.0,78.0,73.0,0.0,Australia,40.0,72.0,69.0,72.0,70.0,0.0,1956
3,0.0,2.0,1956-06-03,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,0.0,Japan,60.0,75.0,76.0,78.0,73.0,0.0,1956
4,2.0,0.0,1956-06-10,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,0.0,Japan,60.0,75.0,76.0,78.0,73.0,0.0,1956
5,1.0,1.0,1956-02-26,Friendlies,Mexico,15.0,79.0,81.0,79.0,77.0,0.0,Costa Rica,25.0,73.0,69.0,73.0,73.0,0.0,1956
6,0.0,0.0,1956-02-28,Friendlies,Argentina,5.0,84.0,86.0,82.0,81.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0,1956
7,0.0,2.0,1956-03-04,Friendlies,Mexico,15.0,79.0,81.0,79.0,77.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0,1956
8,1.0,0.0,1956-03-06,Friendlies,Brazil,2.0,86.0,87.0,85.0,84.0,0.0,Peru,11.0,76.0,78.0,75.0,74.0,0.0,1956
9,4.0,3.0,1956-03-06,Friendlies,Argentina,5.0,84.0,86.0,82.0,81.0,0.0,Costa Rica,25.0,73.0,69.0,73.0,73.0,0.0,1956


In [26]:
# friendlies have already been removed from the dataset when scraping
# (as have futsal, age group tournaments, club games and youth Olympics)
# so no need for a friendly indicator, nor to later remove friendlies
# however, there are club games in there which need to be removed

# indicators for type of game
df.drop(df[[bool(re.compile(r'Club').search(x)) for x in df['cup_name']]].index,inplace=True)
df['qualifier'] = [bool(re.compile(r'Prel|Qual').search(x)) for x in df['cup_name']]
df['finaltourn'] = [not x for x in df['qualifier']]

df.head()

,goals_t1,goals_t2,date,cup_name,long_name_,fifa_ranking_,overall_rating_,att_,mid_,def_,...,long_name2,fifa_ranking2,overall_rating2,att2,mid2,def2,squad_rating2,year,qualifier,finaltourn
0,3.0,6.0,1956-11-11,FIFA WC 1958 Prel. Comp. UEFA,Belgium,3.0,83.0,85.0,83.0,83.0,...,France,7.0,85.0,86.0,84.0,82.0,0.0,1956,True,False
1,5.0,2.0,1956-12-05,FIFA WC 1958 Prel. Comp. UEFA,England,13.0,81.0,83.0,81.0,80.0,...,Denmark,12.0,78.0,76.0,79.0,76.0,0.0,1956,True,False
2,0.0,2.0,1956-11-27,Olympic Football Tournament Melbourne 1956,Japan,60.0,75.0,76.0,78.0,73.0,...,Australia,40.0,72.0,69.0,72.0,70.0,0.0,1956,False,True
3,0.0,2.0,1956-06-03,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,...,Japan,60.0,75.0,76.0,78.0,73.0,0.0,1956,True,False
4,2.0,0.0,1956-06-10,OFT > 1956 > Preliminary Round,Korea Republic,61.0,73.0,71.0,74.0,70.0,...,Japan,60.0,75.0,76.0,78.0,73.0,0.0,1956,True,False


## 4. Feature Engineering

## 5. Models

## 6. Model Validation

## 7. Predictions